In [1]:
import os
%pwd

'/home/towet/Desktop/Visions/OpenProjects/Lung-Colon-Cancer-Histopathological-Images-Classification-Project/research'

In [3]:
os.chdir('../')

In [4]:
%pwd

'/home/towet/Desktop/Visions/OpenProjects/Lung-Colon-Cancer-Histopathological-Images-Classification-Project'

In [5]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen=True)
class DataIngestionConfig:
    root_dir: Path 
    source_URL: Path 
    local_data_file: Path 
    unzip_dir: Path 
    

In [6]:
from LungColonClassifier.constants import *
from LungColonClassifier.utils.common import read_yaml, create_directories

In [7]:
class ConfigurationManager:
    def __init__(self,
                 config_file_path = CONFIG_FILE_PATH,
                 params_file_path = PARAMS_FILE_PATH):
        self.config = read_yaml(config_file_path)
        self.params = read_yaml(params_file_path)

        create_directories([self.config.artifacts_root])
    def get_data_ingestion_config(self) -> DataIngestionConfig:
        config = self.config.data_ingestion 
        create_directories([config.root_dir])
        data_ingestion_config = DataIngestionConfig(
            root_dir=config.root_dir,
            source_URL=config.source_URL,
            local_data_file=config.local_data_file,
            unzip_dir=config.unzip_dir
        )
        return data_ingestion_config

In [8]:
import gdown
import zipfile
import os
from LungColonClassifier import logger

In [9]:
class DataIngestion:
    def __init__(self, config: DataIngestionConfig):
        self.config = config 

    def download_file(self)-> str:
        '''
        Fetch data from the url
        '''

        try: 
            dataset_url = self.config.source_URL
            zip_download_dir = self.config.local_data_file
            os.makedirs("artifacts/data_ingestion", exist_ok=True)
            logger.info(f"Downloading data from {dataset_url} into file {zip_download_dir}")

            file_id = dataset_url.split("/")[-2]
            prefix = 'https://drive.google.com/uc?/export=download&id='
            gdown.download(prefix+file_id,zip_download_dir)

            logger.info(f"Downloaded data from {dataset_url} into file {zip_download_dir}")

        except Exception as e:
            raise e
        
    
    def extract_zip_file(self):
        """
        zip_file_path: str
        Extracts the zip file into the data directory
        Function returns None
        """
        unzip_path = self.config.unzip_dir
        os.makedirs(unzip_path, exist_ok=True)
        with zipfile.ZipFile(self.config.local_data_file, 'r') as zip_ref:
            zip_ref.extractall(unzip_path)

In [10]:
try:
    config = ConfigurationManager()
    ingestion_config = config.get_data_ingestion_config()
    ing = DataIngestion(config=ingestion_config)
    ing.download_file()
    ing.extract_zip_file()
except Exception as e:
    raise e

[2024-02-01 12:05:36,078: INFO: common: yaml file: config/config.yaml loaded successfully]
[2024-02-01 12:05:36,288: INFO: common: yaml file: params.yaml loaded successfully]
[2024-02-01 12:05:36,318: INFO: common: created directory at: artifacts]
[2024-02-01 12:05:36,389: INFO: common: created directory at: artifacts/data_ingestion]
[2024-02-01 12:05:36,390: INFO: 1712959631: Downloading data from https://drive.google.com/file/d/1NDnJPpy-wOgYBSf-kBlaCRpEydVRc7xv/view?usp=sharing into file artifacts/data_ingestion/lung-and-colon-cancer-histopathological-images.zip]


Downloading...
From (original): https://drive.google.com/uc?/export=download&id=1NDnJPpy-wOgYBSf-kBlaCRpEydVRc7xv
From (redirected): https://drive.google.com/uc?/export=download&id=1NDnJPpy-wOgYBSf-kBlaCRpEydVRc7xv&confirm=t&uuid=0d2c1f6e-635c-4e0d-9e64-bf4dcb3ce816
To: /home/towet/Desktop/Visions/OpenProjects/Lung-Colon-Cancer-Histopathological-Images-Classification-Project/artifacts/data_ingestion/lung-and-colon-cancer-histopathological-images.zip
100%|██████████| 1.89G/1.89G [08:19<00:00, 3.78MB/s] 

[2024-02-01 12:13:58,534: INFO: 1712959631: Downloaded data from https://drive.google.com/file/d/1NDnJPpy-wOgYBSf-kBlaCRpEydVRc7xv/view?usp=sharing into file artifacts/data_ingestion/lung-and-colon-cancer-histopathological-images.zip]
